## Kaggel: Digit Recognizer(MNIST)  by Hyperopt  
Kaggle Digit recognizer: https://www.kaggle.com/c/digit-recognizer  
GPyOpt: https://github.com/SheffieldML/GPyOpt  

### Score:
* max_evals= 10, score: -- ( -- mins: NVIDIA GTX1060)
* max_evals= 20, score: -- ( -- mins: NVIDIA GTX1060)
* max_evals=100, score: -- ( -- mins: NVIDIA GTX1060)

In [76]:
import warnings
warnings.filterwarnings('ignore')

import GPy
import GPyOpt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import keras

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# fix random seed
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
seed = 123
rn.seed(seed)
np.random.seed(seed)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(seed)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

## Data preparation: MNIST from Kaggle

In [77]:
train = pd.read_csv('../train.csv')
label = train.label
train = train.drop(['label'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(train, label, test_size=0.2, shuffle=True, random_state=seed)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

## Hyperparameters:

In [78]:
params = [
    {'name': 'Dropout_0',        'type': 'continuous',  'domain': (0.0, 1.0)},
    {'name': 'Dropout_1',        'type': 'continuous',  'domain': (0.0, 1.0)},
    {'name': 'Dropout_2',        'type': 'continuous',  'domain': (0.0, 1.0)},
    {'name': 'Dropout_3',        'type': 'continuous',  'domain': (0.0, 1.0)},
    #{'name': 'BatchNorm_0',      'type': 'discrete',    'domain': (0, 1)},
    #{'name': 'BatchNorm_1',      'type': 'discrete',    'domain': (0, 1)},
    #{'name': 'BatchNorm_2',      'type': 'discrete',    'domain': (0, 1)},
    #{'name': 'BatchNorm_3',      'type': 'discrete',    'domain': (0, 1)},
    {'name': 'Dense_0',          'type': 'discrete',    'domain': (256, 512)},
    {'name': 'Dense_1',          'type': 'discrete',    'domain': (128, 256)},
    {'name': 'validation_split', 'type': 'continuous',  'domain': (0.1, 0.3)},
    {'name': 'batch_size',       'type': 'discrete',    'domain': (8, 16, 32)},
]

In [79]:
# def param(p_name):
#     index = [p['name'] for p in params].index(p_name)
#     return params[index]

# param('Dropout_0')

## CNN Model:

In [80]:
cnt = 0
def cnn_model(x):
    
    def Param(p_name):
        p_index = [p['name'] for p in params].index(p_name)
        p_type = params[p_index]['type']
        if p_type == 'continuous':
            return float(x[:, p_index])
        else:
            return int(x[:, p_index])
    
    initializer = keras.initializers.glorot_uniform(seed=seed)
    
    model = Sequential() 
    model.add(Reshape((28,28,1), input_shape=(784,)))
        
    model.add(Conv2D(32, (5,5), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(Conv2D(32, (5,5), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(Param('Dropout_0'), seed=seed))
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(Param('Dropout_1'), seed=seed))

    model.add(Flatten())
    model.add(Dense(Param('Dense_0'), activation="relu", kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(Dropout(Param('Dropout_2'), seed=seed))
    model.add(Dense(Param('Dense_1'), activation = "relu", kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(Dropout(Param('Dropout_3'), seed=seed))
    
    model.add(Dense(10, activation = "softmax", kernel_initializer=initializer))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-5,verbose=1, cooldown=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
    #checkpointer = ModelCheckpoint(filepath='checkpoint'+str(cnt)+'.hdf5', monitor='val_loss', verbose=1, save_best_only=True)

    hist = model.fit(X_train, Y_train,
                     batch_size=Param('batch_size'),
                     epochs=50,
                     verbose=1,
                     shuffle=True,
                     validation_split=Param('validation_split'),
                     callbacks=[reduce_lr, early_stopping])
    
    #loss, acc = model.evaluate(X_test, Y_test, batch_size=32 , verbose=0)
    val_loss = hist.history['val_loss'][-1]
    val_acc = hist.history['val_acc'][-1]
    
    global cnt
    print(cnt, ': Val_loss:',val_loss, ', Val_acc:', val_acc)
    cnt += 1
    #return {'loss': loss, 'acc': acc,  'model': model, 'hist': hist}
    #return val_loss, val_acc, model, hist
    return val_loss

## Bayesian Optimization:

In [81]:
opt = GPyOpt.methods.BayesianOptimization(f=cnn_model, 
                                          domain=params,
                                          acquisition_type='EI')

Train on 28266 samples, validate on 5334 samples
Epoch 1/50
28266/28266 [==============================] - 21s 741us/step - loss: 0.4898 - acc: 0.8516 - val_loss: 0.0767 - val_acc: 0.9764
Epoch 2/50
28266/28266 [==============================] - 18s 649us/step - loss: 0.1886 - acc: 0.9453 - val_loss: 0.0568 - val_acc: 0.9807
Epoch 3/50
28266/28266 [==============================] - 18s 646us/step - loss: 0.1489 - acc: 0.9574 - val_loss: 0.0522 - val_acc: 0.9844
Epoch 4/50
28266/28266 [==============================] - 18s 646us/step - loss: 0.1248 - acc: 0.9642 - val_loss: 0.0522 - val_acc: 0.9856

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/50
28266/28266 [==============================] - 18s 647us/step - loss: 0.0865 - acc: 0.9755 - val_loss: 0.0338 - val_acc: 0.9910
Epoch 6/50
28266/28266 [==============================] - 18s 647us/step - loss: 0.0714 - acc: 0.9797 - val_loss: 0.0329 - val_acc: 0.9899
Epoch 7/50
28266/28266 [===========

Epoch 11/50
29621/29621 [==============================] - 19s 656us/step - loss: 0.0701 - acc: 0.9781 - val_loss: 0.0297 - val_acc: 0.9892
Epoch 12/50
29621/29621 [==============================] - 19s 656us/step - loss: 0.0726 - acc: 0.9785 - val_loss: 0.0301 - val_acc: 0.9894

Epoch 00012: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 13/50
29621/29621 [==============================] - 19s 656us/step - loss: 0.0678 - acc: 0.9798 - val_loss: 0.0299 - val_acc: 0.9892
Epoch 14/50
29621/29621 [==============================] - 19s 657us/step - loss: 0.0658 - acc: 0.9800 - val_loss: 0.0290 - val_acc: 0.9889
Epoch 15/50
29621/29621 [==============================] - 19s 656us/step - loss: 0.0640 - acc: 0.9803 - val_loss: 0.0302 - val_acc: 0.9892
Epoch 16/50
29621/29621 [==============================] - 19s 657us/step - loss: 0.0681 - acc: 0.9795 - val_loss: 0.0296 - val_acc: 0.9889
Epoch 17/50
29621/29621 [==============================] - 20s 662us/step - loss: 0.0677 - acc:

In [82]:
opt.run_optimization(max_iter=10)

Train on 23520 samples, validate on 10080 samples
Epoch 1/50
23520/23520 [==============================] - 20s 864us/step - loss: 0.5905 - acc: 0.8040 - val_loss: 0.0951 - val_acc: 0.9693
Epoch 2/50
23520/23520 [==============================] - 17s 730us/step - loss: 0.2339 - acc: 0.9251 - val_loss: 0.0794 - val_acc: 0.9749
Epoch 3/50
23520/23520 [==============================] - 18s 752us/step - loss: 0.1824 - acc: 0.9436 - val_loss: 0.0675 - val_acc: 0.9792
Epoch 4/50
23520/23520 [==============================] - 17s 739us/step - loss: 0.1571 - acc: 0.9501 - val_loss: 0.0640 - val_acc: 0.9813
Epoch 5/50
23520/23520 [==============================] - 17s 731us/step - loss: 0.1295 - acc: 0.9596 - val_loss: 0.0601 - val_acc: 0.9811
Epoch 6/50
23520/23520 [==============================] - 17s 727us/step - loss: 0.1204 - acc: 0.9625 - val_loss: 0.0450 - val_acc: 0.9849
Epoch 7/50
23520/23520 [==============================] - 17s 726us/step - loss: 0.1054 - acc: 0.9683 - val_loss: 0.

In [103]:
x_best = opt.x_opt
print([i for i in x_best])

y_best = opt.fx_opt
print(y_best)

[1.0, 0.0, 0.0, 0.0, 256.0, 128.0, 0.1, 16.0]
0.022346920996573143


In [122]:
for i in range(len(x_best)):
    params[i]['domain'] = x_best[i]
params

[{'domain': 1.0, 'name': 'Dropout_0', 'type': 'continuous'},
 {'domain': 0.0, 'name': 'Dropout_1', 'type': 'continuous'},
 {'domain': 0.0, 'name': 'Dropout_2', 'type': 'continuous'},
 {'domain': 0.0, 'name': 'Dropout_3', 'type': 'continuous'},
 {'domain': 256.0, 'name': 'Dense_0', 'type': 'discrete'},
 {'domain': 128.0, 'name': 'Dense_1', 'type': 'discrete'},
 {'domain': 0.1, 'name': 'validation_split', 'type': 'continuous'},
 {'domain': 16.0, 'name': 'batch_size', 'type': 'discrete'}]

In [121]:
x_best[0]

1.0

In [123]:
opt.plot_acquisition()